In [ ]:
import os
import re

#Context-specific
from langchain.document_loaders import PyPDFLoader

#Pipeline needs
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool

#Environment-specific
from langchain.embeddings import VertexAIEmbeddings
from langchain.chat_models import ChatVertexAI

# from google.cloud import aiplatform

### Processing

In [ ]:
llm = ChatVertexAI(temperature=0)
embeddings = VertexAIEmbeddings()

In [ ]:
run = True

if run:

    for file in os.listdir('data/pdfs/'):

        new_name = re.sub(" ", "_", file).lower()
        loader = PyPDFLoader(file)
        splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
        docs = splitter.split_documents(loaded)
        db = Chroma.from_documents(docs, embeddings, collection_name=new_name)
        # print(db._collection_count())


### Chain Implementation

In [ ]:
support_template = """As a Verizon customer service agent, your goal is to provide accurate and helpful information about Verizon products and telecommunications concepts.
You should answer customer questions based on the context provided and avoid making up answers. Remember to provide relevant information and be as concise as possible

Context: {context}
Question: {question}"""

SUPPORT_PROMPT = PromptTemplate(template=support_template, input_variables=["context", "question"])

In [ ]:
support_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": SUPPORT_PROMPT}
)

In [ ]:
#Testing support chain:
support_chain.run("What is 5G?")

In [ ]:
sales_template = """As a Verizon sales representative, your goal is to sell Verizon products and services to customers.
You should answer customer questions based on the context while continually reminding them of products or services offered by Verizon that might respond to their question. 
Remember to provide relevant information and be as concise as possible

Context: {context}
Question: {question}"""

SALES_PROMPT = PromptTemplate(template=sales_template, input_variables=["context", "question"])

In [ ]:
sales_chain = RetrievalQA.from_chain_type(
    llm=ChatVertexAI(temperature=0.7), #ramp up the creativity and schmooze
    chain_type='stuff',
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": SALES_PROMPT}
)

In [ ]:
#Testing sales chain:
support_chain.run("What is the newest Verizon offering?")

### Agent Implementation

In [ ]:
tools = [
    Tool(
        name = "Support",
        func = support_chain.run,
        description = """Useful for when a customer is interested in learning more about Verizon products, services, or other concepts,
                        needs help with understanding. Input should be a fully formed question"""
    ),
    Tool(
        name = "Sales",
        func = sales_chain.run,
        description = """Useful for when a customer indicates they are interested in a Verizon product or service, and that they are
                        willing to buy. Input should be a fully formed question."""
    )
]

In [ ]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run("What will the future of 5G look like for Verizon?")